# BT2101 Assignment 1: Decision Trees

### Kaustubh Jagtap (A0168820B), Group 9

### Import required libraries

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt, log
from __future__ import division
from collections import defaultdict
%matplotlib inline

## Attribute Selection

Functions to help us select the next attribute to split on
1. Entropy calculation
2. Gini Index Calculation
3. Information Gain (based on entropy)
4. Gini Reduction (based on gini index)
5. Decide best feature split

### Function to calculate entropy with any number of sample labels

In [37]:
def entropy(sample_labels):
    """Input: A vector of sample labels e.g. [A,B,A,C,C,C,A,B,B,A,C,B,A,A]. 
       This input has to be for a particular segment of the attribute we are splitting on.
       e.g. if splitting on gender attribute, the vector will be the labels for all males. OR all females. etc   
       Output: A number between 0 and 1, the entropy"""
    
    # Assert np.array
    sample_labels = np.array(sample_labels)
    
    # What if sample_labels are empty
    if sample_labels.size == 0:
        return 0  
    
    # What if all the labels are the same
    class_values = np.unique(sample_labels)
    entropy = 0     
    
    for value in class_values:
        num = len(list(filter(lambda x:x==value, sample_labels)))
        
        if num == 0:
            continue
        
        proportion = num/sample_labels.size
        entropy -= proportion*log(proportion,2)
    
    return entropy

### Function to calculate gini index with any number of sample labels

In [38]:
def gini_index(sample_labels):
    """Input: A vector of sample labels e.g. [A,B,A,C,C,C,A,B,B,A,C,B,A,A]. 
       This input has to be for a particular segment of the attribute we are splitting on.
       e.g. if splitting on gender attribute, the vector will be the labels for all males. OR all females. etc   
       Output: The gini index for this split"""
    
    # Assert np.array
    sample_labels = np.array(sample_labels)
    
    # What if sample_labels are empty
    if sample_labels.size == 0:
        return 0  
    
    # What if all the labels are the same
    class_values = np.unique(sample_labels)# Sample labels/classes; Usually (0,1), sometimes (-1,1)
    collective_sum = 0   ## initialize the (1 - gini_index)
    
    for value in class_values:
        num = len(list(filter(lambda x:x==value, sample_labels)))
                
        proportion = num/sample_labels.size
        collective_sum += proportion**2
    
    gini_index = 1 - collective_sum
    
    return gini_index

### Function to calculate information gain

In [39]:
def info_gain(curr_df, output, selected_attribute):
    '''
    This function is used to calculate the difference in the starting entropy and new entropy
    when a given tree node is splitted by a given feature.
    
    Inputs:
    1) curr_df: Samples in the current tree node before making split on the attribute (Pandas Dataframe)
    2) output: Name of the output column
    3) selected_attribute: Name of the feature/ attribute used to split the current tree node.
    
    Outputs:
    1) measure_weighted: The weighted average of the measure. 
    2) info_gain: How much information is gained (entropy_start - entropy_weighted) if the current tree node is splitted 
    by the attribute.
    3) subsamples (dict): All labels for the different values within selected attribute. 
    
    '''
    
    # calculate overall entropy of dataframe
    entropy_start = entropy(curr_df[output])
    
    # list of segments/ values that our attribute can take e.g. [male, female]
    attribute_groups = curr_df[selected_attribute].unique()
    
    # initialize weighted entropy
    entropy_weighted = 0   
    
    # Split samples by attribute values into subsamples
    subsamples = defaultdict()
    
    for value in attribute_groups:
        
        # initialise dataframe pertaining to this value
        labels_for_value = curr_df[curr_df[selected_attribute] == value]
      
        # Consolidate every group into subsamples. Key of dict = value (e.g. male/female). Value of dict = dataframe for that value
        subsamples[value] = labels_for_value
        
        # Calculate the entropy for each value
        entropy_for_this_value = entropy(labels_for_value[output])

        entropy_weighted += len(labels_for_value)*entropy_for_this_value/len(curr_df)
    
    info_gain = entropy_start - entropy_weighted


    return (entropy_weighted, info_gain, subsamples)  

### Function to calculate reduction in gini index

In [40]:
def gini_reduction(curr_df, output, selected_attribute):
    '''
    This function is used to calculate the difference in the starting gini index and new gini index
    when a given tree node is splitted by a given feature.
    
    Inputs:
    1) curr_df: Samples in the current tree node before making split on the attribute (Pandas Dataframe)
    2) output: Name of the output column
    3) selected_attribute: Name of the feature/ attribute used to split the current tree node.
    
    Outputs:
    1) gini_end: The weighted average of the gini index for this attribute. 
    2) reduction: How much reduction in impurity of dataset (gini_start - gini_end) if the current tree node is splitted 
    by the attribute.
    3) subsamples (dict): All labels for the different values within selected attribute. 
    
    '''
    
    # calculate overall entropy of dataframe
    gini_start = gini_index(curr_df[output])
    
    # list of segments/ values that our attribute can take e.g. [male, female]
    attribute_groups = curr_df[selected_attribute].unique()
    
    # initialize gini index for this attribute
    gini_end = 0   
    
    # Split samples by attribute values into subsamples
    subsamples = defaultdict()
    
    for value in attribute_groups:
        
        # initialise dataframe pertaining to this value
        labels_for_value = curr_df[curr_df[selected_attribute] == value]
      
        # Consolidate every group into subsamples. Key of dict = value (e.g. male/female). Value of dict = dataframe for that value
        subsamples[value] = labels_for_value
        
        # Calculate the entropy for each value
        gini_for_this_value = gini_index(labels_for_value[output])

        gini_end += len(labels_for_value)*gini_for_this_value/len(curr_df)
    
    reduction = gini_start - gini_end


    return (gini_end, reduction, subsamples)  

### Function to decide best feature split

In [46]:
def best_feature_split(curr_df, output, attributes, method):
  
  '''
  This function is used to determine the best attribute to split based on maximized information gain.
  
  Inputs:
  1) curr_df: Samples in the current tree node before making split on the attribute (Pandas Dataframe)
  2) output: Name of the output column
  3) attributes: A list of feature names
  4) method: Method employed to evaluate the improvement in the purity of dataset 
  (method is either info_gain or gini_reduction)

  Outputs:
  1) best_attribute: The best feature which is used to do binary splitting
  2) best_improvement: The greatest gini_reduction or information_gain (depending on method employed)
  3) best_subsamples (dict): Data samples of best feature      

  '''
  
  # Initialise best feature, best info_gain/reduction, best subsamples
  best_attribute = None
  best_improvement = 0
  best_subsamples = None
  
  # Number of rows in the data samples
  num_instances = float(len(curr_df))
  
  # Loop through attributes and find the best attribute to split on
  for attribute in attributes:    
    
    current_split = method(curr_df, output, attribute)
    
    improvement = current_split[1]    # either info_gain or gini reduction
    subsamples = current_split[2]
    
    # Check if attribute is better
    if improvement >= best_improvement:
      best_attribute, best_improvement, best_subsamples = (attribute, improvement, subsamples)
     
    
  
  return (best_attribute, best_improvement, best_subsamples) 

## Stopping Conditions for pre-pruning

1. The samples' labels in the current node are the same
2. All the features have already been used for split
3. The current tree has already reached maximum depth max_depth
4. The number of samples in the current node is lower than minimum number min_number
5. The information gain for the current split is lower than a threshold min_infogain

### Stopping Condition 1: The samples' labels in the current node are the same

In [47]:
def stop_1(node_labels):
    '''This function is used to verify whether stopping condition 1 is satisfied.
    Inputs:
    1) node_labels: The samples' labels in the current node
    
    Outputs:
    1) True if they are all the same, False if otherwise
    
    '''
    
    # numpy array
    node_labels = np.array(node_labels)
    
    # Empty labels
    if len(node_labels) == 0:
        return True
    
    if len(np.unique(node_labels)) == 1:
        print("Stopping Condition 1: The samples' labels in the current node are the same")
        return True
    else:
        return False

### Stopping Condition 2: All the features have already been used for split

In [48]:
def stop_2(features):
    '''This function is used to verify whether stopping condition 2 is satisfied.
    Inputs:
    1) features: A list of feature names
    
    Outputs:
    1) True if the feature list is empty, False if otherwise
    
    '''
    
    if len(features) == 0 or features == None:
        print("Stopping Condition 2: All the features have already been used for split")
        return True
    else:
        return False  

### Stopping Condition 3: The current tree has already reached maximum depth max_depth

In [49]:
def stop_3(tree_depth, max_depth):
    '''This function is used to verify whether stopping condition 3 is satisfied.
    Inputs:
    1) tree_depth: The depth of the current tree
    2) max_depth: Maximum tree depth
    
    Outputs:
    1) True if the current depth reaches maximum depth, False if otherwise
    
    '''
    
    if tree_depth >= max_depth:
        print("Stopping Condition 3: The current tree has already reached maximum depth")
        return True
    else:
        return False 

### Stopping Condition 4: Number of samples in the current node is lower than  min_number

In [50]:
def stop_4(samples, min_number):
    '''This function is used to verify whether stopping condition 4 is satisfied.
    Inputs:
    1) samples: Data samples in the current node (Pandas DataFrame)
    2) min_number: Minimum number of node size
    
    Outputs:
    1) True if sample size is smaller than the minimum number, False if otherwise
    
    '''
    
    if samples.size <= min_number:
        print("Stopping Condition 4: The number of samples in the current node is lower than minimum number")
        return True
    else:
        return False  

### Stopping Condition 5: The information gain for the current split is lower than threshold min_infogain¶

In [51]:
# info_gain(samples, output, feature) -> information gain, left, right
# best_feature_split(samples, output, features) -> feature, information gain, left, right
def stop_5(info_gain, min_infogain):
    '''This function is used to verify whether stopping condition 5 is satisfied.
    Inputs:
    1) info_gain: Information gain after this best split
    2) min_infogain: Minimum information gain
    
    Outputs:
    1) True if information gain after this best splitting is smaller than the minimum number, False if otherwise
    
    '''
    
    if info_gain <= min_infogain:
        print("Stopping Condition 5: The information gain for the current split is lower than a threshold")
        return True
    else:
        return False  

## Building the decision tree

### Function to assign predicted label based on highest count (majority)

In [52]:
def majority_vote(output_labels):
    '''
    This function is used to get predicted label based on "Majority Voting" criterion for the current leaf node.     
    Inputs:
    1) output_labels: Outputs (labels) in this leaf node, such as [A, B, A, C, C, B]
    
    Outputs:
    1) prediction: Predicted label for this leaf node (e.g. A)
    
    '''
    
    # numpy array
    output_labels = np.array(output_labels)
    
    # Empty label
    if output_labels.size == 0:
        return None
    
    # Count output labels (0/-1 or 1)
    values = np.unique(output_labels)
    
    # Initialise a list containing a set of tuples that includes the value and len(value)
    lst = list(map(lambda x:(x, len(output_labels[output_labels == x])), values))
    
    # Identify the value with the highest count
    # Prediction based on "Majority Voting" criterion
    
    return max(lst, key=lambda x:x[1])[0]

### Function to generate classification tree, with any number of labels

In [65]:
def ClassificationTree(curr_df, output, attributes, step, tree_depth, max_depth, min_number, min_improvement, method):
    '''This function is used to build a classification tree in a recursive way.
       Remember how you build a binary tree in the previous C++ and Data Structure courses).
       
    Inputs:
    1) curr_df: Samples in the current tree node before making split on the feature (Pandas Dataframe)
    2) output: Name of the output column
    3) attributes: A list of feature names
    4) step: The current binary split step
    5) tree_depth: The depth of the current tree
    6) max_depth: Maximum depth this tree can grow
    7) min_number: Minimum number of node size
    8) min_improvement: Minimum information gain or gini reduction
    9) method: Method to measure improvement in the purity of dataset (info_gain or gini_reduction)
    
    Outputs:
    1) tree_nodes: Nested tree nodes, which are stored and shown in nested dictionary type
    
    Return format:
    
    {'best_attribute': <> ,
     'label': <>,
     'subtree 0': {'best_attribute': <>,
                   'label': <>,
                   'subtree 0': {'best_attribute': <>,
                                 'label': '<>',
                                 'sub_trees': None},
                   'subtree 1': {'best_attribute': <>,
                                 'label': <>,
                                 'subtree 0': {'best_attribute': <>,
                                               'label': <>,
                                               'sub_trees': None},
                                 'subtree 1': {'best_attribute': <>,
                                               'label': <>,
                                               'sub_trees': None},
                                 'subtree 2': {'best_attribute': <>,
                                               'label': <>,
                                               'sub_trees': None}},
                   'subtree 2': {'best_attribute': <>,
                                 'label': <>,
                                 'sub_trees': None}}

    
    '''
    # Available attributes that we can split on
    available_attributes = list(attributes)
    
    # Output labels in the current tree node
    labels = curr_df[output]
    
    print ("----------------------------------------------------------------------------")
    print ("----------------------------------------------------------------------------")
    print ("Step %s: Current tree depth is %s. Current tree node has %s data points and %s instances." 
           % (step, tree_depth, curr_df.size, curr_df.shape[0]))
    
    # Verify whether stopping conditions 1-4 are satisfied. If satisfied, return a leaf_node
    if stop_1(labels) or stop_2(available_attributes) or stop_3(tree_depth, max_depth) or stop_4(curr_df, min_number):
      return {
          'label': majority_vote(labels),
          'sub_trees': None, # dict
          'best_attribute': None
      }
    
    # If pass stopping conditions 1-4, then do best splitting
    best_split = best_feature_split(curr_df, output, available_attributes, method) # outputs: best_attribute, best_improvement, best_subsamples (dict)
    best_attribute, best_improvement, best_subsamples = best_split
    
    # Verify whether stopping condition 5 is satisfied. If satisfied, return a leaf node
    if stop_5(best_improvement, min_improvement):
      return {
          'label': majority_vote(labels),
          'sub_trees': None, # dict
          'best_attribute': None
      }
    # If pass stopping condition 5, then move on
    step += 1
    
    # print out message for logging purpose
    message = "Step {}: Binary split on {}. Sizes are ".format(step, best_attribute)
      
    for attribute_value in best_subsamples:
      num_instances = len(best_subsamples[attribute_value])
      message += "[{}: {}]".format(attribute_value, num_instances)
    
    print(message)
    
    # Remove this attribute if this attribute is used for split
    available_attributes.remove(best_attribute)
    
    # Do split on tree in a recursive way
    sub_trees = [] # initialise an empty list to store the recursive calls
    for attribute_value in best_subsamples:
      this_split = ClassificationTree(best_subsamples[attribute_value],
                                      output,
                                      available_attributes,
                                      step+1,
                                      tree_depth+1,
                                      max_depth,
                                      min_number,
                                      min_improvement,
                                      method
                                     )
      sub_trees.append(this_split)
    
    # Initialise final_dict to unpack the sub_trees list
    final_dict = {'label': None,
                  'best_attribute': best_attribute
                 }
    
    for i in range(len(sub_trees)):
      this_subtree = "subtree {}".format(i)
      final_dict[this_subtree] = sub_trees[i]
    
    return final_dict

## Evaluation of model

### Function to calculate misclassification rate before running the model

In [ ]:
def misclassification_rate(actual, predicted):
    '''
    This function is used to assess the effectiveness of each split based on misclassification percentage.
    Used during post-pruning.
    
    Inputs:
    1) Actual: Vector of actual labels
    2) Predicted: The predicted label assigned to that node
    
    Output: A percentage value, the error rate
    '''
    
    misclass_count = 0
    for label in actual:
        if label != predicted:
            misclass_count += 1
            
    return misclass_count/len(actual)

### Function to calculate misclassification rate after running the model

In [67]:
def misclassification_rate(actual, predicted):
    '''
    This function is used to assess the effectiveness of the prediction of the model, via misclassification metric
    Inputs:
    1) Actual: Vector of actual labels
    2) Predicted: Vector of predicted labels
    
    Output: A percentage value, the error rate
    '''
    misclass_count = 0
    for i in range(len(actual)):
        if actual[i] != predicted[i]:
            misclass_counter += 1
    
    return misclass_counter/len(actual)
    

### 5.1 What if the functions are continuous? Explain.

If the features are continuous, we will not be able to effectively use decision trees, as they are more suitable for dealing with categorical variables. Some alternatives would be to either use the nearest neighbour classification, regression methods, or neural nets (e.g. for time series). On the other hand, if we want to still use decision trees, we would need to discretize the continuous attributes. 

One way to do this would be to assign buckets (e.g. weights from 0-10, 10-20 etc…). This falls under global discretization, and the resultant categorical variable can be treated as a new label regardless of the choice of our data mining algorithm. However, one drawback of this is that the data loses its meaning and out predictions are the buckets instead of the actual values.

Another way to still carry out decision tree classification would be to perform local discretisation, an example of which is rounding off to the nearest integer. Although this can increase the accuracy of the predictions, it introduces too many features and can lead to eventual overfitting.



### 5.2 What if the output is continuous? Explain.

#### CART
If the output is continuous, then one solution would be to build a regression tree. In this process, we choose a predictor and a cutpoint, and partition the space such that the loss is minimized. A simple way to measure loss is to calculate the residual sum of squares (RSS). 

After obtaining this predictor and cutpoint, we recursively perform binary splitting on one of the 2 spaces and stop when some pre-defined termination condition is satisfied.
